## 최신 기출동형 모의고사 - 제3회 기출동형 모의고사
### 1. 머신러닝 (50점)

#### (1) 데이터를 탐색하고 탐색 결과를 제시하시오.

In [ ]:
import pandas as pd
import numpy as np 
hotel = pd.read_csv('../data/hotel_bookings.csv')
hotel

# 결측치 확인
hotel.isna().sum()

In [ ]:
hotel.info()

In [ ]:
hotel.deposit_type.describe()

In [ ]:
hotel.groupby(hotel['deposit_type']).mean()['adr']

#### (2) 결측치를 탐색하고 대체 방법 및 근거를 제시하시오.

In [ ]:
# 결측치 확인
hotel.isna().sum()

In [ ]:
#결측치 비율 확인
print('lead time 결측치 비율: {}%'.format(round(5 /len(hotel)*100, 2)))
print('is repeated guest 결측치 비율: {}%'.format(round(358/len(hotel)*100, 2)))
print('adr 결측치 비율: {}%'.format(round(1063 /len(hotel)*100, 2)))

In [ ]:
hotel_nonull=hotel
## lead time 변수의 결측치는 1%가 되지 않음
## 제거한다
hotel_nonull.dropna(subset=['lead_time'], axis =0, inplace =True)

## is_repeated_guest 변수는 binary이기 때문에
## 최빈값으로 대체한다
hotel_nonull['is_repeated_guest'].fillna(0)

## adr 변수는 연속형 변수이기 때문에 평균대치법을 사용한다.
## 다만, 보증금 유형에 따라 adr이 다르기 때문에
## adr그룹별 평균을 이용해 결측값을 대체한다.
fill_mean_func = lambda g: g.fillna(g.mean())
hotel_nonull=hotel_nonull.groupby('deposit_type').apply(fill_mean_func)

hotel_nonull.index =hotel_nonull.index.droplevel(0 )
hotel_nonull.sort_index(inplace =True )

hotel_nonull.isna().sum()

#### (4) 데이터 불균형을 시각화하여 식별하고 불균형 판단근거를 작성하시오.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
font_path ='C:/Windows/Fonts/NGULIM.TTF'
font = font_manager.FontProperties(fname =font_path).get_name()
rc('font', family =font)

sns.countplot(x='is_canceled', data =hotel_nonull)
plt.title('데이터 불균형 시각화', fontsize =14)
plt.show()
ratio0 = round(len(hotel_nonull[hotel_nonull['is_canceled']==0])/len(hotel_nonull)*100, 2)
ratio1 = round(len(hotel_nonull[hotel_nonull['is_canceled']==1])/len(hotel_nonull)*100, 2)
print('0 비율: {}%'.format(ratio0))
print('1 비율: {}%'.format(ratio1))

#### (6) 기법을 선정한 이유를 작성하고, 원데이터를 포함해 3개의 데이터 세트를 구성하시오.

In [ ]:
# 먼저 hotel_null 데이터의 object 형태 변수를 one_hot_encoding함
hotel_nonull=pd.get_dummies(hotel_nonull)

from imblearn.over_sampling import RandomOverSampler, SMOTE
import time
# 원본 데이터 (hotel_nonull)
X = hotel_nonull[hotel_nonull.columns.difference(['is_canceled'])]
y = hotel_nonull['is_canceled']

start = time.time() # 시작 시간 저장
# Random Oversampling
ros = RandomOverSampler(random_state =42)
X_ro, y_ro = ros.fit_resample(X, y)
print('time :', time.time() - start) # 현재시각 - 시작시간 = 실행 시간

In [ ]:
start = time.time() # 시작 시간 저장
# SMOTE
sm = SMOTE(random_state =42)
X_sm, y_sm = ros.fit_resample(X, y)
print('time :', time.time() - start) # 현재시각 - 시작시간 = 실행 시간

#### (7) 오버 샘플링 데이터와 원데이터를 사용해 정확도 측면 모델 하나와 속도 측면의 모델 하나를 선정하고, 그 이유를 설명하시오.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

start = time.time() # 시작 시간 저장
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.2, stratify =y, random_state =100)
clf = RandomForestClassifier(n_estimators =100, min_samples_split =10)
clf.fit(X_train, y_train)
print('train 정확도 :', clf.score(X_train, y_train), '\n')
pred=clf.predict(X_test)
print(classification_report(y_test, pred))
print('time :', time.time() - start) # 현재시각 - 시작시간 = 실행 시간

In [ ]:
start = time.time() # 시작 시간 저장
X_ro_train, X_ro_test, y_ro_train, y_ro_test = train_test_split(X_ro, y_ro, test_size =0.2, stratify =y_ro, random_state =100)
clf_ro = RandomForestClassifier(n_estimators =100, min_samples_split=10, random_state =100)
clf_ro.fit(X_ro_train, y_ro_train)
print('train 정확도 :', clf_ro.score(X_ro_train, y_ro_train), '\n')
pred_ro=clf_ro.predict(X_ro_test)
print(classification_report(y_ro_test, pred_ro))
print('time :', time.time() - start) # 현재시각 - 시작시간 = 실행 시간

In [ ]:
start = time.time() # 시작 시간 저장
X_sm_train, X_sm_test, y_sm_train, y_sm_test = train_test_split(X_sm, y_sm, test_size =0.2, stratify =y_sm, random_state =100)
clf_sm = RandomForestClassifier(n_estimators =100, min_samples_split=10, random_state =100)
clf_sm.fit(X_sm_train, y_sm_train)
print('train 정확도 :', clf_sm.score(X_sm_train, y_sm_train), '\n')
pred_sm=clf_sm.predict(X_sm_test)
print(classification_report(y_sm_test, pred_sm))
print('time :', time.time() - start) # 현재시각 - 시작시간 = 실행 시간

In [ ]:
raw=0 +1.7477023601531982
ro=0.03391218185424805+3.019350051879883
sm=0.0389096736907959+3.2345712184906006
print('원본 데이터세트의 모델링 수행속도: ','\t', raw)
print('RO 데이터세트의 모델링 수행속도: ','\t', ro)
print('SMOTE 데이터세트의 모델링 수행속도: ','\t', sm) 

### 2. 통계분석 (50점)

#### 1. 공장에서는 시제품의 농도(%)가 60이라고 주장하며 품질관리팀에서 10개의 샘플을 뽑았다. 유의수준 5%에서 다음을 검정하시오.

In [ ]:
x =[52 ,50 ,62 ,75 ,26 ,45 ,62 ,35 ,57 ,14 ]

import numpy as np
moe=5
a=0.05
std = np.std(x)
print(std)

In [ ]:
from scipy import stats
(stats.t.ppf(q =.05 ,df =9 )*std /5 )**2

In [ ]:
from scipy import stats
stats.wilcoxon(pd.Series(x)-60)

#### 2. 코로나 시계열 데이터로 다음을 수행하시오.

In [ ]:
import pandas as pd
covid = pd.read_csv('../data/서울특별시 코로나19.csv')
print (covid.head())
covid1 = covid[covid.columns.difference(['날짜'])]

In [ ]:
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.arima_process import ArmaProcess
def acf(x, n_lags):
    return sm.tsa.stattools.acf(x, nlags =n_lags)

# Max ACF lags
n_lags =785
lag_arr = np.repeat(n_lags, covid1.shape[1])
acf_list = list(map(acf, covid1.transpose().to_numpy(), lag_arr))
acf_df = pd.DataFrame(acf_list).transpose()
acf_df.columns = covid1.columns

acf_df

In [ ]:
acf_df =acf_df.T
acf_df.head()

In [ ]:
import scipy.cluster.hierarchy as sch
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
font_path ='C:/Windows/Fonts/NGULIM.TTF'
font = font_manager.FontProperties(fname =font_path).get_name()
rc('font', family =font)

plt.figure(figsize=(15, 5))
label = acf_df.index
dend1=sch.linkage(acf_df, method ='average')
cutoff =0.5 *max(dend1[:,2])
dend_res1=sch.dendrogram(dend1, color_threshold=cutoff, labels=label)
plt.show()

#### 3. 사회과학, 자연과학, 공학 세 개 학과의 평점조사표를 보고 학과와 성적이 관계있는지 검정하시오.

In [ ]:
사회 = [16, 30, 12]
자연 = [12, 20, 3]
공학 = [18, 13, 14]
table = pd.DataFrame({'사회과학': 사회,
                     '자연과학': 자연,
                     '공학': 공학},
                     index=['3.5~4.5', '2.5~3.5', '1.5~2.5'])
print (table)

In [ ]:
from scipy import stats
stats.chi2_contingency(observed=table)